# Train & Align NN Acoustic Model
Align already while training.

https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [1]:
%run ../prongen/hmm_pron.py --in-jupyter
%run ../acmodel/plot.py
%matplotlib ipympl

%run ../acmodel/matrix.py
%run ../acmodel/praat_ifc.py

%run ../acmodel/hmm_acmodel.py


device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"
print(f"Using {device} device")


%run ../acmodel/nn_acmodel.py

hmm_pron.py library - generate Czech pron HMM. Included to this notebook.


Using cuda device


## Get training data
We aligned Czech CommonVoice train set using an ultra-prinmitive HMM/GMM. Let's use it as a starting point.

In [2]:
#import pandas as pd
#pd.set_option('display.max_colwidth', None)

#f = pd.read_csv("nn_train.tsv", sep="\t", keep_default_na=False)
#df = pd.read_csv("nn_train_g2.tsv", sep="\t", keep_default_na=False)
#df = pd.read_csv("nn_train_g3.tsv", sep="\t", keep_default_na=False)
#df = pd.read_csv("nn_train_g4.tsv", sep="\t", keep_default_na=False)


In [3]:
#df

In [4]:
#hmms = []
#for wav, sentence, targets in list(zip(df.wav.values, df.sentence.values, df.targets.values)):
#    hmm = HMM(sentence, wav=wav, use_DA=True)
#    hmm.targets = targets
#    hmms.append(hmm)

infile = "mega2_training_0000.tsv"

#df = pd.read_csv("nn_train_g5.tsv", sep="\t", keep_default_na=False)  # USED JUST AS A TEMPLATE FOR SAVED ALIGNMENTS
df = pd.read_csv(infile, sep="\t", keep_default_na=False)

    
#hmms = get_training_hmms("nn_train_g5.tsv", derivatives=3)
hmms = get_training_hmms(infile, derivatives=3)




In [5]:
#b_set = sorted({*"".join([hmm.b for hmm in hmms ])}) # make sorted set of all phone names in the training set

all_mfcc, all_targets, b_set = collect_training_material(hmms)

out_size = len(b_set)
in_size = hmms[0].mfcc.size(1)

" ".join(b_set), out_size, in_size

('? A E G H N O Z a b c d e f g h j k l m n o p r s t u v y z | á é ó ú ý č ď ň Ř ř š ť Ž ž',
 45,
 52)

In [6]:
#all_targets = "".join([hmm.targets for hmm in hmms])
#train_len = len(all_targets)

In [7]:
#all_mfcc = torch.cat([hmm.mfcc for hmm in hmms]).double().to(device)
#assert all_mfcc.size()[0]==train_len

## Setup PyTorch training tools

In [8]:
##import os
#import torch
#from torch import nn
#from torch.utils.data import DataLoader
#from torch.utils.data import Dataset
##from torchvision import datasets, transforms

In [9]:
model = NeuralNetwork(in_size, out_size).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=52, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=45, bias=True)
  )
)


In [10]:
#from torch.utils.data import DataLoader

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [12]:
#torch.save(model.state_dict(), 'model_weights_g4_20.pth')

In [ ]:
training_data = SpeechDataset(all_mfcc, all_targets, b_set) # initial alignment


for mega_epoch in range(100): # change to 100
    print(f"============ Training Mega Epoch {mega_epoch} =============")

    all_targets = "".join([hmm.targets for hmm in hmms])  # collect alignments
    training_data.all_targets = all_targets  # just update the object with new targets

    train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True) # new dataloader for this alignment

    train_n_epochs(train_dataloader, optimizer, model, criterion, 5) # change to 10

    print('Interupted training for re-alignment...')

    model.eval() # switch to evaluation mode


    for idx, hmm in enumerate(hmms):
        if idx%1000==0:
            print(f"Align {idx}")
    
        alp = viterbi_log_align_nn(hmm, model, b_set)
        hmm.intervals = backward_log_alignment_pass_intervals(hmm, alp) # also modifies alp
        hmm.indices = i = alp.max(1).indices
        s = "".join([hmm.b[ii] for ii in i])
        hmm.troubling = troubling_alignmet(s)
        hmm.targets = "".join([hmm.b[ii] for ii in i])

        
        
    df['targets'] = [hmm.targets for hmm in hmms]

    filename_base = f"mega3_training_{'%04d' % mega_epoch}"

    torch.save(model.state_dict(), filename_base+".pth")
    df.to_csv(filename_base+".tsv", sep="\t", index=False)


============ Training Mega Epoch 0 =============
[1, 20000] loss: 1.066
[1, 40000] loss: 0.991
[1, 60000] loss: 0.963
[2, 20000] loss: 0.939
[2, 40000] loss: 0.931
[2, 60000] loss: 0.925
[3, 20000] loss: 0.913
[3, 40000] loss: 0.911
[3, 60000] loss: 0.910
[4, 20000] loss: 0.899
[4, 40000] loss: 0.899
[4, 60000] loss: 0.897
[5, 20000] loss: 0.889
[5, 40000] loss: 0.890
[5, 60000] loss: 0.890
Interupted training for re-alignment...
Align 0
Align 1000
Align 2000
Align 3000
Align 4000
Align 5000
Align 6000
Align 7000
Align 8000
Align 9000
Align 10000
============ Training Mega Epoch 1 =============
[1, 20000] loss: 0.863
[1, 40000] loss: 0.864
[1, 60000] loss: 0.864
[2, 20000] loss: 0.852
[2, 40000] loss: 0.858
[2, 60000] loss: 0.859
[3, 20000] loss: 0.850
[3, 40000] loss: 0.851
[3, 60000] loss: 0.853
[4, 20000] loss: 0.845
[4, 40000] loss: 0.847
[4, 60000] loss: 0.849
[5, 20000] loss: 0.840
[5, 40000] loss: 0.844
[5, 60000] loss: 0.846
Interupted training for re-alignment...
Align 0
Align

In [ ]:
STOP

https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html